# L7: Build a Crew to Tailor Job Applications

In [ ]:
from crewai import Agent, Task, Crew

In [ ]:
import os
import getpass

# Prompt the user for input (hidden)
api_key = getpass.getpass("Enter the openai api key to set it as environment variable 'OPENAI_API_KEY': ")
api_base = getpass.getpass("Enter the openai api base to set it as environment variable 'OPENAI_API_BASE': ")
# serper_api_key = getpass.getpass("Enter the SERPER_API_KEY for efficient search: ")
os.environ["SERPER_API_KEY"] = getpass.getpass("Enter the serper api key to set it as environment variable 'SERPER_API_KEY': ")

os.environ["OPENAI_API_KEY"] = api_key
os.environ["OPENAI_API_BASE"] = api_base
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

## crewAI Tools

In [3]:
from crewai_tools import (
  FileReadTool,
  ScrapeWebsiteTool,
  MDXSearchTool,
  SerperDevTool
)

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
read_resume = FileReadTool(file_path='./fake_resume.md')
semantic_search_resume = MDXSearchTool(mdx='./fake_resume.md')

/opt/homebrew/lib/python3.10/site-packages/pydantic/_internal/_config.py:295: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)
/opt/homebrew/lib/python3.10/site-packages/pydantic/_internal/_fields.py:192: UserWarning: Field name "schema" in "DatabricksQueryToolSchema" shadows an attribute in parent "BaseModel"
  warnings.warn(
/opt/homebrew/lib/python3.10/site-packages/pydantic/_internal/_generate_schema.py:502: UserWarning: <built-in function callable> is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(
/opt/homebrew/lib/python3.10/site-packa

- Uncomment and run the cell below if you wish to view `fake_resume.md` in the notebook.

In [ ]:
# from IPython.display import Markdown, display
# display(Markdown("./fake_resume.md"))

## Creating Agents

In [5]:
# Agent 1: Researcher
researcher = Agent(
    role="Tech Job Researcher",
    goal="Make sure to do amazing analysis on "
         "job posting to help job applicants",
    tools = [scrape_tool, search_tool],
    verbose=True,
    backstory=(
        "As a Job Researcher, your prowess in "
        "navigating and extracting critical "
        "information from job postings is unmatched."
        "Your skills help pinpoint the necessary "
        "qualifications and skills sought "
        "by employers, forming the foundation for "
        "effective application tailoring."
    )
)

In [6]:
# Agent 2: Profiler
profiler = Agent(
    role="Personal Profiler for Engineers",
    goal="Do increditble research on job applicants "
         "to help them stand out in the job market",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "Equipped with analytical prowess, you dissect "
        "and synthesize information "
        "from diverse sources to craft comprehensive "
        "personal and professional profiles, laying the "
        "groundwork for personalized resume enhancements."
    )
)

In [7]:
# Agent 3: Resume Strategist
resume_strategist = Agent(
    role="Resume Strategist for Engineers",
    goal="Find all the best ways to make a "
         "resume stand out in the job market.",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "With a strategic mind and an eye for detail, you "
        "excel at refining resumes to highlight the most "
        "relevant skills and experiences, ensuring they "
        "resonate perfectly with the job's requirements."
    )
)

In [8]:
# Agent 4: Interview Preparer
interview_preparer = Agent(
    role="Engineering Interview Preparer",
    goal="Create interview questions and talking points "
         "based on the resume and job requirements",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "Your role is crucial in anticipating the dynamics of "
        "interviews. With your ability to formulate key questions "
        "and talking points, you prepare candidates for success, "
        "ensuring they can confidently address all aspects of the "
        "job they are applying for."
    )
)

## Creating Tasks

In [9]:
# Task for Researcher Agent: Extract Job Requirements
research_task = Task(
    description=(
        "Analyze the job posting URL provided ({job_posting_url}) "
        "to extract key skills, experiences, and qualifications "
        "required. Use the tools to gather content and identify "
        "and categorize the requirements."
    ),
    expected_output=(
        "A structured list of job requirements, including necessary "
        "skills, qualifications, and experiences."
    ),
    agent=researcher,
    async_execution=True
)

In [10]:
# Task for Profiler Agent: Compile Comprehensive Profile
profile_task = Task(
    description=(
        "Compile a detailed personal and professional profile "
        "using the GitHub ({github_url}) URLs, and personal write-up "
        "({personal_writeup}). Utilize tools to extract and "
        "synthesize information from these sources."
    ),
    expected_output=(
        "A comprehensive profile document that includes skills, "
        "project experiences, contributions, interests, and "
        "communication style."
    ),
    agent=profiler,
    async_execution=True
)

- You can pass a list of tasks as `context` to a task.
- The task then takes into account the output of those tasks in its execution.
- The task will not run until it has the output(s) from those tasks.

In [11]:
# Task for Resume Strategist Agent: Align Resume with Job Requirements
resume_strategy_task = Task(
    description=(
        "Using the profile and job requirements obtained from "
        "previous tasks, tailor the resume to highlight the most "
        "relevant areas. Employ tools to adjust and enhance the "
        "resume content. Make sure this is the best resume even but "
        "don't make up any information. Update every section, "
        "inlcuding the initial summary, work experience, skills, "
        "and education. All to better reflrect the candidates "
        "abilities and how it matches the job posting."
    ),
    expected_output=(
        "An updated resume that effectively highlights the candidate's "
        "qualifications and experiences relevant to the job."
    ),
    output_file="tailored_resume.md",
    context=[research_task, profile_task],
    agent=resume_strategist
)

In [12]:
# Task for Interview Preparer Agent: Develop Interview Materials
interview_preparation_task = Task(
    description=(
        "Create a set of potential interview questions and talking "
        "points based on the tailored resume and job requirements. "
        "Utilize tools to generate relevant questions and discussion "
        "points. Make sure to use these question and talking points to "
        "help the candiadte highlight the main points of the resume "
        "and how it matches the job posting."
    ),
    expected_output=(
        "A document containing key questions and talking points "
        "that the candidate should prepare for the initial interview."
    ),
    output_file="interview_materials.md",
    context=[research_task, profile_task, resume_strategy_task],
    agent=interview_preparer
)

## Creating the Crew

In [13]:
job_application_crew = Crew(
    agents=[researcher,
            profiler,
            resume_strategist,
            interview_preparer],

    tasks=[research_task,
           profile_task,
           resume_strategy_task,
           interview_preparation_task],

    verbose=True
)

## Running the Crew

- Set the inputs for the execution of the crew.

In [14]:
job_application_inputs = {
    'job_posting_url': 'https://jobs.lever.co/AIFund/6c82e23e-d954-4dd8-a734-c0c2c5ee00f1?lever-origin=applied&lever-source%5B%5D=AI+Fund',
    'github_url': 'https://github.com/joaomdmoura',
    'personal_writeup': """Noah is an accomplished Software
    Engineering Leader with 18 years of experience, specializing in
    managing remote and in-office teams, and expert in multiple
    programming languages and frameworks. He holds an MBA and a strong
    background in AI and data science. Noah has successfully led
    major tech initiatives and startups, proving his ability to drive
    innovation and growth in the tech industry. Ideal for leadership
    roles that require a strategic and innovative approach."""
}

In [15]:
### this execution will take a few minutes to run
result = job_application_crew.kickoff(inputs=job_application_inputs)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: dbe2ca7e-b697-4f3c-81c6-e8e1b7e4821c                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: eca9657e-7bd7-4610-b7c2-f69df92bcd7d
       Status: Executing Task...

🚀 Crew: crew
├── 📋 Task: eca9657e-7bd7-4610-b7c2-f69df92bcd7d
│      Status: Executing Task...
└── 📋 Task: c58c1133-934c-4925-8ec9-38a0cef1b178
       Status: Executing Task...

🚀 Crew: crew
├── 📋 Task: eca9657e-7bd7-4610-b7c2-f69df92bcd7d
│      Status: Executing Task...
└── 📋 Task: c58c1133-934c-4925-8ec9-38a0cef1b178
       Status: Executing Task...
    └── 🤖 Agent: Tech Job Researcher
            Status: In Progress

# Agent: Tech Job Researcher
## Task: Analyze the job posting URL provided (https://jobs.lever.co/AIFund/6c82e23e-d954-4dd8-a734-c0c2c5ee00f1?lever-origin=applied&lever-source%5B%5D=AI+Fund) to extract key skills, experiences, and qualifications required. Use the tools to gather content and identify and categorize the requirements.


...



# Agent: Engineering Interview Preparer
## Final Answer: 
Interview Questions and Talking Points for Noah Williams:

1. Can you elaborate on your experience with managing remote and in-office teams, and how you ensure effective collaboration and performance enhancement across different locations?
2. You have a strong background in software development and various programming languages. Could you share a specific project where you utilized your expertise in Ruby, Python, JavaScript, TypeScript, or Elixir to solve a complex technical challenge?
3. Tell us about your involvement in integrating cutting-edge AI technologies and scalable vector databases at DataKernel. How did this initiative contribute to the company's revenue growth?
4. As the Founder & CEO of InnovPet, how did you establish and lead a startup focused on IoT solutions for pet care? What were the key challenges you faced during the product development phase?
5. In your role as the Engineering Manager at EliteDevs, how did

🚀 Crew: crew
├── 📋 Task: eca9657e-7bd7-4610-b7c2-f69df92bcd7d
│      Assigned to: Tech Job Researcher
│      Status: ✅ Completed
├── 📋 Task: c58c1133-934c-4925-8ec9-38a0cef1b178
│      Assigned to: Personal Profiler for Engineers
│      Status: ✅ Completed
│   ├── 🤖 Agent: Tech Job Researcher
│   │       Status: In Progress
│   │   └── 🧠 Thinking...
│   ├── 🤖 Agent: Tech Job Researcher
│   │       Status: ✅ Completed
│   │   ├── 🔧 Used Search the internet with Serper (1)
│   │   ├── 🔧 Using Search the internet with Serper (1)
│   │   ├── 🔧 Used Read a file's content (1)
│   │   └── 🔧 Used Read website content (2)
│   └── 🤖 Agent: Personal Profiler for Engineers
│           Status: ✅ Completed
│       ├── 🔧 Used Read website content (3)
│       ├── 🔧 Used Search the internet with Serper (2)
│       └── 🔧 Used Read a file's content (2)
├── 📋 Task: b3755cb6-ff7f-4edd-95bf-b65587f5ca52
│      Assigned to: Resume Strategist for Engineers
│      Status: ✅ Completed
│   └── 🤖 Agent: Resume Strategist for Engineers
│           Status: ✅ Completed
│       └── 🔧 Used Search a MDX's content (1)
└── 📋 Task: 2934b0f0-8a0d-4b14-b423-7631bea513a6
       Status: Executing Task...
    └── 🤖 Agent: Engineering Interview Preparer
            Status: ✅ Completed
        └── 🔧 Used Search a MDX's content (2)

🚀 Crew: crew
├── 📋 Task: eca9657e-7bd7-4610-b7c2-f69df92bcd7d
│      Assigned to: Tech Job Researcher
│      Status: ✅ Completed
├── 📋 Task: c58c1133-934c-4925-8ec9-38a0cef1b178
│      Assigned to: Personal Profiler for Engineers
│      Status: ✅ Completed
│   ├── 🤖 Agent: Tech Job Researcher
│   │       Status: In Progress
│   │   └── 🧠 Thinking...
│   ├── 🤖 Agent: Tech Job Researcher
│   │       Status: ✅ Completed
│   │   ├── 🔧 Used Search the internet with Serper (1)
│   │   ├── 🔧 Using Search the internet with Serper (1)
│   │   ├── 🔧 Used Read a file's content (1)
│   │   └── 🔧 Used Read website content (2)
│   └── 🤖 Agent: Personal Profiler for Engineers
│           Status: ✅ Completed
│       ├── 🔧 Used Read website content (3)
│       ├── 🔧 Used Search the internet with Serper (2)
│       └── 🔧 Used Read a file's content (2)
├── 📋 Task: b3755cb6-ff7f-4edd-95bf-b65587f5ca52
│      Assigned to: Resume Strategist for Engineers
│      Status: ✅ Completed
│   └── 🤖 Agent: Resume Strategist for Engineers
│           Status: ✅ Completed
│       └── 🔧 Used Search a MDX's content (1)
└── 📋 Task: 2934b0f0-8a0d-4b14-b423-7631bea513a6
       Assigned to: Engineering Interview Preparer
       Status: ✅ Completed
    └── 🤖 Agent: Engineering Interview Preparer
            Status: ✅ Completed
        └── 🔧 Used Search a MDX's content (2)

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 2934b0f0-8a0d-4b14-b423-7631bea513a6                                                                     │
│  Agent: Engineering Interview Preparer                                                                          │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: dbe2ca7e-b697-4f3c-81c6-e8e1b7e4821c                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

- Dislplay the generated `tailored_resume.md` file.

In [16]:
from IPython.display import Markdown, display
display(Markdown("./tailored_resume.md"))

# Noah Williams

## Personal Summary
Accomplished Software Engineering Leader with 18 years of experience in the technology industry. Well-suited for senior executive roles, particularly in companies seeking leadership with a blend of technical and strategic expertise. Specializes in managing diverse remote and in-office teams, demonstrating proficiency in various programming languages and frameworks. Holds an MBA and possesses a strong background in AI and data science.

## Skills
- Software Development
- Team Management
- Programming Languages: Ruby, Python, JavaScript, TypeScript, Elixir
- Front End Frameworks
- AI and Data Science
- Process Innovation

## Project Experiences
### DataKernel: Senior Software Engineering Manager (remote) — 2019 - 2022
- Directed engineering strategy and operations, collaborating closely with C-level executives to shape the company's technological trajectory.
- Managed diverse teams across multiple time zones, enhancing team performance through a transparent and respectful work environment.
- Recruited, mentored, and retained top talent, fostering a culture of continuous improvement and high performance.

### DriveAI: Senior Software Engineer (remote) — 2015 - 2016
- Developed and optimized a central API, improving functionality for a large engineering team and users, enhancing system performance and user satisfaction.
- Implemented critical enhancements, including advanced caching strategies for reduced response times and system loads.

### BetCraft: CTO — 2013 - 2015
- Led technology strategy post-Series A funding, guiding the company through technological advancement and network expansion.
- Enhanced platform performance and market reach through strategic initiatives and partnerships.
- Proficient in building products using React and Angular.

## Contributions
Instrumental in driving technological advancements and fostering innovation in all companies worked with. Strategic initiatives consistently lead to positive outcomes, contributing significantly to organizational growth and success.

## Interests
Keen interest in speaking at conferences, remote work, data science, game development, and contributing to open source projects. Stays updated with industry trends and explores new technologies.

## Communication Style
Known for transparent and collaborative communication style. Values mutual respect and transparency, fostering an environment for constructive feedback and open dialogue.

## Education
- MBA in Information Technology, London Business School
- B.Sc. in Computer Science, University of Edinburgh
- Data Science Specialization Certificate, Coursera (Johns Hopkins University)
- Advanced Leadership Techniques Certificate, University of London

- Dislplay the generated `interview_materials.md` file.

In [17]:
display(Markdown("./interview_materials.md"))

Interview Questions and Talking Points for Noah Williams:

1. Can you elaborate on your experience with managing remote and in-office teams, and how you ensure effective collaboration and performance enhancement across different locations?
2. You have a strong background in software development and various programming languages. Could you share a specific project where you utilized your expertise in Ruby, Python, JavaScript, TypeScript, or Elixir to solve a complex technical challenge?
3. Tell us about your involvement in integrating cutting-edge AI technologies and scalable vector databases at DataKernel. How did this initiative contribute to the company's revenue growth?
4. As the Founder & CEO of InnovPet, how did you establish and lead a startup focused on IoT solutions for pet care? What were the key challenges you faced during the product development phase?
5. In your role as the Engineering Manager at EliteDevs, how did you strategize and coordinate inter-departmental efforts to achieve common goals? Can you provide an example of a successful project that resulted from this coordination?
6. At PrintPack, you developed a high-performance engineering team that led to a significant revenue increase. What strategies did you implement to enhance team performance and drive business growth?
7. As a Senior Software Engineer at DriveAI, you optimized the central API for improved functionality. Can you walk us through the process of identifying bottlenecks and implementing solutions to enhance system performance?
8. In your position as the CTO at BetCraft, how did you lead the technology strategy post-Series A funding and expand the company's market reach? What technologies or strategies did you implement to improve platform performance and user experience?
9. How do you stay updated with industry trends and new technologies, and how do you incorporate this knowledge into your work and leadership style?
10. Can you provide an example of a situation where your transparent and collaborative communication style had a positive impact on team dynamics and project outcomes?

These questions and discussion points are tailored to Noah Williams' professional journey, skills, and experiences, aligning with his background as a Software Engineering Leader and AI specialist.

In [ ]:
# https://docs.crewai.com/introduction